In [1]:
import pandas as pd
pd.read_csv?

In [10]:
filename = "data/season_1/training_data/order_data/order_data_2016-01-01"
columns = ['order_id', 'driver_id', 'passenger_id', 'start_district_hash', 'dest_district_hash', 'price', 'time']

df = pd.read_csv(filename, sep = "\t", names = columns)
df.head(2)

,order_id,driver_id,passenger_id,start_district_hash,dest_district_hash,price,time
0,97ebd0c6680f7c0535dbfdead6e51b4b,dd65fa250fca2833a3a8c16d2cf0457c,ed180d7daf639d936f1aeae4f7fb482f,4725c39a5e5f4c188d382da3910b3f3f,3e12208dd0be281c92a6ab57d9a6fb32,24,2016-01-01 13:37:23
1,92c3ac9251cc9b5aab90b114a1e363be,c077e0297639edcb1df6189e8cda2c3d,191a180f0a262aff3267775c4fac8972,82cc4851f9e4faa4e54309f8bb73fd7c,b05379ac3f9b7d99370d443cfd5dcc28,2,2016-01-01 09:47:54


In [ ]:
count = df[['order_id', 'driver_id']].groupby('order_id').count()
count = count['driver_id']

# Orders picked up by more than one driver?
print(sum(count > 1))
# Yes..? Surprising.

In [11]:
# Turns out there are duplicate and almost-duplicate entries. Let's keep the last ones.
dup = df.duplicated(['order_id', 'driver_id', 'passenger_id', 'time'], keep = 'last')
df = df[~dup]

In [28]:
# Orders picked up by more than one driver?
print(sum(count > 1))
# No.

# Proportion of orders no picked up by a driver
p = sum(count == 0)/len(count)
print("{:.1%}".format(p))
# It appears the gap is simply the number of orders not picked up.

0
35.0%


In [62]:
# Compute time slot
# The first time slot on Jan. 23rd, 2016; one day is uniformly divided into 144 ten minute time slots.

df['datetime'] = pd.to_datetime(data['time'])
df['date'] = pd.to_datetime(df.datetime.dt.date)
df['time'] = df.datetime.dt.time
# df = df.drop('datetime', axis = 1)

df['timeslot'] = (df['datetime'] - df['date']).astype('timedelta64[m]')//10
df.head(2)

,order_id,driver_id,passenger_id,start_district_hash,dest_district_hash,price,time,datetime,date,timeslot
0,97ebd0c6680f7c0535dbfdead6e51b4b,dd65fa250fca2833a3a8c16d2cf0457c,ed180d7daf639d936f1aeae4f7fb482f,4725c39a5e5f4c188d382da3910b3f3f,3e12208dd0be281c92a6ab57d9a6fb32,24,09:47:54,2016-01-01 09:47:54,2016-01-01,58
1,92c3ac9251cc9b5aab90b114a1e363be,c077e0297639edcb1df6189e8cda2c3d,191a180f0a262aff3267775c4fac8972,82cc4851f9e4faa4e54309f8bb73fd7c,b05379ac3f9b7d99370d443cfd5dcc28,2,18:24:02,2016-01-01 18:24:02,2016-01-01,110
